In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from PIL import Image

%matplotlib inline

In [2]:
batch_size = 128

In [3]:
def data_preparer():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        
    test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)
    
    return train_loader, test_loader

In [4]:
class Net(nn.Module):
    def __init__(self, net_dim=32):
        super().__init__()
        self.conv1 = nn.Conv2d(3, net_dim*2, kernel_size=7, padding=1, bias=False)
        self.batch1 = nn.BatchNorm2d(net_dim*2)
        
        self.conv2 = nn.Conv2d(net_dim*2, net_dim*4, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool2d((2,2))
        self.batch2 = nn.BatchNorm2d(net_dim*4)
        
        self.conv3 = nn.Conv2d(net_dim*4, net_dim*8, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool2d((2, 2))
        self.batch3 = nn.BatchNorm2d(net_dim*8)
        self.drop = nn.Dropout()
        
        self.flat = nn.Flatten(1,3)
                
        self.linear = nn.Linear(9216, 10)
            
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.batch1(self.conv1(x)))
        x = self.relu(self.batch2(self.pool1(self.conv2(x))))
        x = self.drop(self.relu(self.batch3(self.pool2(self.conv3(x)))))
        
        x = self.flat(x)
        
        x = self.linear(x)
        
        return x

In [5]:
def log_metrics(cost, epoch, iteration, train_predictions, train_labels, test_predictions, test_labels, batch_size):
    train_correct = ((train_predictions > 0.90) * train_labels).sum()
    train_accuracy = train_correct / batch_size * 100

    test_correct = ((test_predictions > 0.90) * test_labels).sum()
    test_accuracy = test_correct / batch_size * 100
        
    print("Epoch %d, iteration %d, cost equals %f" % (epoch, iteration, cost))
    print("Train accuracy %f, Test accuracy %f" % (train_accuracy, test_accuracy))

In [6]:
def training_loop(train_loader, test_loader, net, criterion, optimizer, num_epochs, device='cuda:0'):
        print("Training Started")
        iteration = 0
        net.train()
        
        for epoch in range(1, num_epochs+1):
            for x_train, labels_train in train_loader:
                                
                x_train = x_train.to(device)
                labels_train = labels_train.to(device)
                                
                # Updating the Network
                optimizer.zero_grad()
                x_predictions = net(x_train)  
                
                cost = criterion(x_predictions, labels_train)
                cost.backward()
                optimizer.step()
        
        print("Training Ended")
        return net

In [7]:
classifier = Net().to('cuda:0')

optimizer = optim.Adam(classifier.parameters(), lr=0.000275, weight_decay=0.003)

loss = nn.CrossEntropyLoss()

train_loader, test_loader = data_preparer()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
net = training_loop(train_loader, test_loader, classifier, loss, optimizer, 100)               

Training Started
Training Ended


In [9]:
net.eval()
device = "cuda:0"

correct = 0
test_length = 0
for x_test, labels_test in test_loader:
    test_length += len(labels_test)
    
    x_test = x_test.to(device)
    labels_test = nn.functional.one_hot(labels_test, 10).to(device)
    
    with torch.no_grad():
        prediction_test = net(x_test)

    num_correct_batch = ((prediction_test > 0.90) * labels_test).sum()
    correct += num_correct_batch
    
model_accuracy = correct / test_length * 100
print("\n\nThe model had a final test accuracy of %f" % model_accuracy)



The model had a final test accuracy of 96.470001
